# Generalized Network Analysis Tutorial - Non-Canonical and Non-Proteic Residues

This tutorial will show how to prepare systems with non-canonical amino acid residues and non-proteic residues such as small ligands, sugars, and lipids.

If you have not gone through the first part of this tutorial, we strongly encourage you to start with the "Generalized Network Analysis Tutorial - Step 1" notebook.

The Dynamic Network Analysis package requires a description of **node groups**. These can be trivial to obtain, such as when a non-canonical amino acid is represented by its alpha-carbon. However, for large lipid molecules in a cell membrane, or long and flexible ligands, one may wish to describe the residue using multiple node groups.

This tutorial will explore specialized functions and visualization tools that will help you prepare your system for Dynamic Network Analysis.

We will once more analyze the MD trajectory for the OMP decarboxylase system and then move on to a more complicated example using a large lipid molecule.


In [2]:
# Load the python package
import os
import dynetan as dna
from dynetan.toolkit import formatNodeGroups, showNodeGroups

# For visualization
import nglview    as nv

# Create the object that processes MD trajectories.
dnap = dna.proctraj.DNAproc()

In [3]:
workDir = "./TutorialData/"

# PSF file name
psfFile = os.path.join(workDir, "decarboxylase.0.psf")

# DCD file name
dcdFiles = [os.path.join(workDir, "decarboxylase.1.short.dcd")]

In [4]:
dnap.loadSystem(psfFile,dcdFiles)

Here, we use MDAnalysis selection language to select a single valine residue. Note that we usually remove all hydrogen atoms from a system before Dynamical Network Analysis. We do this because node movement already captures most hydrogen-mediated interactions (like H-bonds), and keeping them would drastically increase the computational cost for contact detection.

In [5]:
# atmGrp = dnap.workU.select_atoms("resid 11 and resname VAL")
atmGrp = dnap.workU.select_atoms("resid 11 and resname VAL and (not (name H* or name [123]H*))")
atmGrp

<AtomGroup with 7 atoms>

These are all the names for atoms in our selection:

In [6]:
print(list(atmGrp.names))

['N', 'CA', 'CB', 'CG1', 'CG2', 'C', 'O']


### DyNetAn syntax

Now that we know all atoms in our residue, we can decide how to split them into node groups. We usually take the alpha-carbon as the node for simple amino acid residues, assingning all backbone and side chain atoms to a single node group.

To facilitate this process, we can use the function `formatNodeGroups` from `dynetan.toolkit` :

In [7]:
formatNodeGroups(atmGrp, ["CA"])


        You can copy and paste the following lines into your notebook to define
        the node group(s) for your new residue.
        
usrNodeGroups["VAL"] = {}
usrNodeGroups["VAL"]["CA"] = {'CG1', 'CG2', 'CA', 'CB', 'C', 'O', 'N'}


As the function output indicates, we can copy and paste the code to indicate how we wish to analyze our system.

In [8]:
usrNodeGroups = {}

usrNodeGroups["VAL"] = {}
usrNodeGroups["VAL"]["CA"] = {'O', 'CB', 'CA', 'CG1', 'C', 'CG2', 'N'}

usrNodeGroups

{'VAL': {'CA': {'C', 'CA', 'CB', 'CG1', 'CG2', 'N', 'O'}}}

# Visual check

Here, we use [NGLviewer](https://nglviewer.org/ngl/api/manual/index.html) to create an interactive visualization of our residue and to check how nodes and node groups are assigned.

1. First, let's create a full visualization of the residue, labeling all nodes and groups. This will highlight any atom(s) left out of our selections.
    - Nodes will show up with **atom names** in <font color='green'>**GREEN**</font>, node group atoms will be labelled with **atom names** in **BLACK**, and unselected atoms will be labelled with their **atom names** in <font color='red'>**RED**</font>.

In [10]:
# http://nglviewer.org/ngl/api/typedef/index.html#static-typedef-LabelRepresentationParameters

w = nv.show_mdanalysis(atmGrp)

w._remote_call("setSize", target="Widget", args=["800px", "600px"])
w.parameters = dict(theme='light')

w.clear_representations()
w.add_representation(repr_type="ball+stick", selection="all")

showNodeGroups(w, atmGrp, usrNodeGroups)

w

NGLWidget(max_frame=19)

### Divide a residue into multiple groups:

2. Now, let's pretend we want to break up all valine residues into two node groups, one centered at the alpha-carbon and another at the beta-carbon, like so:

In [11]:
formatNodeGroups(atmGrp, 
                           ["CA","CB"], 
                           [
                               ['C', 'O', 'N', 'CA'],
                               ['CB', 'CG1', 'CG2']
                           ]
                          )


        You can copy and paste the following lines into your notebook to define
        the node group(s) for your new residue.
        
usrNodeGroups["VAL"] = {}
usrNodeGroups["VAL"]["CA"] = {'C', 'CA', 'N', 'O'}
usrNodeGroups["VAL"]["CB"] = {'CB', 'CG1', 'CG2'}


In [12]:
usrNodeGroups = {}

usrNodeGroups["VAL"] = {}
usrNodeGroups["VAL"]["CA"] = {'CA', 'O', 'C', 'N'}
usrNodeGroups["VAL"]["CB"] = {'CG1', 'CG2', 'CB'}

usrNodeGroups

{'VAL': {'CA': {'C', 'CA', 'N', 'O'}, 'CB': {'CB', 'CG1', 'CG2'}}}

In [14]:
w = nv.show_mdanalysis(atmGrp)

w._remote_call("setSize", target="Widget", args=["800px", "600px"])
w.parameters = dict(theme='light')

w.clear_representations()
w.add_representation(repr_type="ball+stick", selection="all")

showNodeGroups(w, atmGrp, usrNodeGroups)

w

NGLWidget(max_frame=19)

### Sanity check

3. Finally, let's pretend we accidentally removed one atom from our selections, like so:

(You will notice the nitrogen atom's label will show up in red, to indicate it does not belong to any atom group)

In [15]:
usrNodeGroups = {}

usrNodeGroups["VAL"] = {}
# usrNodeGroups["VAL"]["CA"] = ['C', 'O', 'N', 'CA']
usrNodeGroups["VAL"]["CA"] = {'CA', 'O', 'C'}
usrNodeGroups["VAL"]["CB"] = {'CG1', 'CG2', 'CB'}

usrNodeGroups

{'VAL': {'CA': {'C', 'CA', 'O'}, 'CB': {'CB', 'CG1', 'CG2'}}}

In [16]:
w = nv.show_mdanalysis(atmGrp)

w._remote_call("setSize", target="Widget", args=["800px", "600px"])
w.parameters = dict(theme='light')

w.clear_representations()
w.add_representation(repr_type="ball+stick", selection="all")

showNodeGroups(w, atmGrp, usrNodeGroups)

w

NGLWidget(max_frame=19)

# Cardiolipin: Dividing a large lipid into node groups

For the second part of this tutorial, we will tackle a more complex case of a large flexible residue: The Cardiolipin. This is a major component of the inner mitochondrial membrane, essential for animal and plant cells, and commonly found in bacterial membranes.

Its large and flexible structure creates a problem for network analysis since representing the entire residue through a single atom would likely lead us to miss important interactions between proteins and cardiolipin's phosphatidic acid and alkyl moieties.

Let's start by loading the system:

In [17]:
workDir = "./TutorialData/NonCanonical/"

# PSF file name
psfFile = os.path.join(workDir, "clip.psf")

# DCD file name
dcdFiles = [os.path.join(workDir, "clip.pdb")]

In [18]:
dnap.loadSystem(psfFile,dcdFiles)

/home/melomcr/anaconda3/envs/DyNetAn_DevEnv_MDA2/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


Now we can use MDAnalysis to access atom names. As usual, we remove hydrogen atoms from our selection:

In [19]:
atmGrp = dnap.workU.select_atoms("resname TOCL2 and (not (name H* or name [123]H*))")
atmGrp

<AtomGroup with 100 atoms>

These are all the names for the 100 heavy atoms (or non-hydrogen atoms) in our selection:

In [20]:
print(list(atmGrp.names))

['C3', 'P3', 'OP33', 'OP34', 'OP31', 'OP32', 'C31', 'C2', 'OG12', 'C1', 'P1', 'OP13', 'OP14', 'OP11', 'OP12', 'C11', 'C12', 'O12', 'CA1', 'OA1', 'CA2', 'C13', 'O13', 'CB1', 'OB1', 'CB2', 'C32', 'O32', 'CC1', 'OC1', 'CC2', 'C33', 'O33', 'CD1', 'OD1', 'CD2', 'CA3', 'CA4', 'CA5', 'CA6', 'CA7', 'CA8', 'CA9', 'CA10', 'CA11', 'CA12', 'CA13', 'CA14', 'CA15', 'CA16', 'CA17', 'CA18', 'CB3', 'CB4', 'CB5', 'CB6', 'CB7', 'CB8', 'CB9', 'CB10', 'CB11', 'CB12', 'CB13', 'CB14', 'CB15', 'CB16', 'CB17', 'CB18', 'CC3', 'CC4', 'CC5', 'CC6', 'CC7', 'CC8', 'CC9', 'CC10', 'CC11', 'CC12', 'CC13', 'CC14', 'CC15', 'CC16', 'CC17', 'CC18', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12', 'CD13', 'CD14', 'CD15', 'CD16', 'CD17', 'CD18']


We can take a look at this simplified representation here:

In [21]:
w = nv.show_mdanalysis(atmGrp)

w._remote_call("setSize", target="Widget", args=["800px", "600px"])
w.parameters = dict(theme='light')

w.clear_representations()
w.add_representation(repr_type="ball+stick", selection="all")

w

NGLWidget()

Now we need to create the selections for each node group.

Looking at our structure, we can define a few atoms to be prominent nodes in the system. We will select the central carbon atom in the glycerol group and the central carbon atom in each phspholipidic headgroup, creating a total of **five** node groups for our residue:
- C2, CA1, CB1, CC1, CD1

In [22]:

nodes  = ["C2", "CA1", "CB1", "CC1", "CD1"]


We will now use the list of atom names to create each node group.

['C3', 'P3', 'OP33', 'OP34', 'OP31', 'OP32', 'C31', 'C2', 'OG12', 'C1', 'P1', 'OP13', 'OP14', 'OP11', 'OP12', 'C11', 'C12', 'O12', 'CA1', 'OA1', 'CA2', 'C13', 'O13', 'CB1', 'OB1', 'CB2', 'C32', 'O32', 'CC1', 'OC1', 'CC2', 'C33', 'O33', 'CD1', 'OD1', 'CD2', 'CA3', 'CA4', 'CA5', 'CA6', 'CA7', 'CA8', 'CA9', 'CA10', 'CA11', 'CA12', 'CA13', 'CA14', 'CA15', 'CA16', 'CA17', 'CA18', 'CB3', 'CB4', 'CB5', 'CB6', 'CB7', 'CB8', 'CB9', 'CB10', 'CB11', 'CB12', 'CB13', 'CB14', 'CB15', 'CB16', 'CB17', 'CB18', 'CC3', 'CC4', 'CC5', 'CC6', 'CC7', 'CC8', 'CC9', 'CC10', 'CC11', 'CC12', 'CC13', 'CC14', 'CC15', 'CC16', 'CC17', 'CC18', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12', 'CD13', 'CD14', 'CD15', 'CD16', 'CD17', 'CD18']

The atoms are organized in a way that can help us split them into contiguous sections of the structure. By splitting the above list into segments, the pattern becomes clear:

- 'C3', 'P3', 'OP33', 'OP34', 'OP31', 'OP32', 'C31', 'C2', 'OG12', 'C1', 'P1', 'OP13', 'OP14', 'OP11', 'OP12', 'C11'

- 'C12', 'O12', 'CA1', 'OA1', 'CA2', 

- 'C13', 'O13', 'CB1', 'OB1', 'CB2', 

- 'C32', 'O32', 'CC1', 'OC1', 'CC2', 

- 'C33', 'O33', 'CD1', 'OD1', 'CD2', 

- 'CA3', 'CA4', 'CA5', 'CA6', 'CA7', 'CA8', 'CA9', 'CA10', 'CA11', 'CA12', 'CA13', 'CA14', 'CA15', 'CA16', 'CA17', 'CA18',

- 'CB3', 'CB4', 'CB5', 'CB6', 'CB7', 'CB8', 'CB9', 'CB10', 'CB11', 'CB12', 'CB13', 'CB14', 'CB15', 'CB16', 'CB17', 'CB18', 

- 'CC3', 'CC4', 'CC5', 'CC6', 'CC7', 'CC8', 'CC9', 'CC10', 'CC11', 'CC12', 'CC13', 'CC14', 'CC15', 'CC16', 'CC17', 'CC18', 

- 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12', 'CD13', 'CD14', 'CD15', 'CD16', 'CD17', 'CD18'

In [23]:
groups = [
    ['C3', 'P3', 'OP33', 'OP34', 'OP31', 'OP32', 'C31', 'C2', 'OG12', 'C1', 'P1', 'OP13', 'OP14', 'OP11', 'OP12', 'C11'],
    ['C12', 'O12', 'CA1', 'OA1', 'CA2', 'CA3', 'CA4', 'CA5', 'CA6', 'CA7', 'CA8', 'CA9', 'CA10', 'CA11', 'CA12', 'CA13', 'CA14', 'CA15', 'CA16', 'CA17', 'CA18'],
    ['C13', 'O13', 'CB1', 'OB1', 'CB2', 'CB3', 'CB4', 'CB5', 'CB6', 'CB7', 'CB8', 'CB9', 'CB10', 'CB11', 'CB12', 'CB13', 'CB14', 'CB15', 'CB16', 'CB17', 'CB18'],
    ['C32', 'O32', 'CC1', 'OC1', 'CC2', 'CC3', 'CC4', 'CC5', 'CC6', 'CC7', 'CC8', 'CC9', 'CC10', 'CC11', 'CC12', 'CC13', 'CC14', 'CC15', 'CC16', 'CC17', 'CC18'],
    ['C33', 'O33', 'CD1', 'OD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12', 'CD13', 'CD14', 'CD15', 'CD16', 'CD17', 'CD18']
]

formatNodeGroups(atmGrp, nodes, groups)


        You can copy and paste the following lines into your notebook to define
        the node group(s) for your new residue.
        
usrNodeGroups["TOCL2"] = {}
usrNodeGroups["TOCL2"]["C2"] = {'C3', 'OG12', 'OP31', 'OP34', 'P1', 'OP11', 'C11', 'C31', 'OP14', 'OP12', 'C1', 'C2', 'OP32', 'OP13', 'P3', 'OP33'}
usrNodeGroups["TOCL2"]["CA1"] = {'CA3', 'CA15', 'CA6', 'CA17', 'C12', 'CA10', 'CA13', 'OA1', 'CA11', 'CA16', 'CA4', 'CA7', 'O12', 'CA2', 'CA1', 'CA14', 'CA18', 'CA12', 'CA5', 'CA8', 'CA9'}
usrNodeGroups["TOCL2"]["CB1"] = {'CB9', 'CB2', 'O13', 'CB6', 'CB10', 'CB13', 'C13', 'OB1', 'CB15', 'CB8', 'CB12', 'CB7', 'CB4', 'CB5', 'CB3', 'CB18', 'CB11', 'CB17', 'CB1', 'CB14', 'CB16'}
usrNodeGroups["TOCL2"]["CC1"] = {'C32', 'CC11', 'CC13', 'CC6', 'CC7', 'O32', 'OC1', 'CC12', 'CC10', 'CC1', 'CC16', 'CC17', 'CC4', 'CC5', 'CC3', 'CC2', 'CC14', 'CC8', 'CC9', 'CC15', 'CC18'}
usrNodeGroups["TOCL2"]["CD1"] = {'CD5', 'C33', 'CD11', 'OD1', 'CD17', 'CD16', 'CD2', 'CD7', 'CD8', 'CD4', 'CD10', 'CD14

In [24]:
usrNodeGroups["TOCL2"] = {}
usrNodeGroups["TOCL2"]["C2"] = {'OP31', 'OP33', 'C2', 'C31', 'OP14', 'P1', 'OG12', 'OP34', 'C1', 'C11', 'OP12', 'P3', 'C3', 'OP32', 'OP13', 'OP11'}
usrNodeGroups["TOCL2"]["CA1"] = {'CA17', 'CA15', 'CA18', 'CA10', 'CA4', 'C12', 'CA12', 'CA7', 'CA2', 'OA1', 'CA5', 'CA6', 'CA9', 'CA16', 'CA13', 'CA1', 'O12', 'CA3', 'CA8', 'CA14', 'CA11'}
usrNodeGroups["TOCL2"]["CB1"] = {'CB2', 'CB9', 'O13', 'CB3', 'CB12', 'CB16', 'CB18', 'CB8', 'C13', 'CB1', 'OB1', 'CB11', 'CB17', 'CB15', 'CB5', 'CB6', 'CB10', 'CB13', 'CB14', 'CB7', 'CB4'}
usrNodeGroups["TOCL2"]["CC1"] = {'CC14', 'OC1', 'CC13', 'CC17', 'CC5', 'CC12', 'CC6', 'CC18', 'CC15', 'CC3', 'CC1', 'CC7', 'CC10', 'CC16', 'O32', 'CC9', 'C32', 'CC8', 'CC2', 'CC4', 'CC11'}
usrNodeGroups["TOCL2"]["CD1"] = {'CD14', 'CD12', 'CD9', 'O33', 'CD16', 'CD4', 'CD18', 'CD13', 'CD1', 'OD1', 'CD5', 'CD8', 'CD10', 'CD6', 'CD15', 'CD2', 'CD7', 'C33', 'CD17', 'CD3', 'CD11'}

Now we can visualize the selection:

In [26]:
w = nv.show_mdanalysis(atmGrp)

w._remote_call("setSize", target="Widget", args=["800px", "600px"])
w.parameters = dict(theme='light')

w.clear_representations()
w.add_representation(repr_type="ball+stick", selection="all")

showNodeGroups(w, atmGrp, usrNodeGroups)

w

NGLWidget()

One more, notice how nodes will show up with **atom names** in <font color='green'>**GREEN**</font> and node group atoms will be labelled with **atom names** in **BLACK**.

We can also examine each group individually by giving the function `showNodeGroups` an extra argument:

In [28]:
w = nv.show_mdanalysis(atmGrp)

w._remote_call("setSize", target="Widget", args=["800px", "600px"])
w.parameters = dict(theme='light')

w.clear_representations()
w.add_representation(repr_type="ball+stick", selection="all")

showNodeGroups(w, atmGrp, usrNodeGroups, node_atm_sel="CC1")

w

NGLWidget()

# The End